In [8]:
!pip install numpy
!pip install ordered_set


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [1]:
import numpy as np
import os
import hashlib
from ordered_set import OrderedSet

In [3]:
#set of shingles without duplicates, in order of insertion
#generate the same hash values for equivalent shingles each time this function is called
#(using hashlib, generate hexadecimal values)
def Shingling(k, file_path):
    with open(file_path, 'r') as file:
        words = [word for line in file for word in line.strip().split()]
    ordered_shingles = OrderedSet()
    #shingles = []
    for i in range(len(words)-k):
        # need to convert sublist to string in order to use encode:
        shingle = ' '.join(words[i:i+k-1])
        """
        hash_value = hashlib.md5(shingle.encode()).hexdigest() #hash(tuple(shingle))
        ordered_shingles.add(hash_value)
        shingles.append(hash_value)
        """
        hash_value_0 = hashlib.md5(shingle.encode())
        # converting to integer within range (0,2^(32)-1)
        hash_value = int(hash_value_0.hexdigest(), 16) % (2**32)
        ordered_shingles.add(hash_value)
    
    return ordered_shingles

In [4]:

def CompareSets(hashed_shingle_set1, hashed_shingle_set2):
    # Compute the jaccard similarity of the hash values(shingles) in the sets:
    jacc = hashed_shingle_set1.intersection(hashed_shingle_set2)/hashed_shingle_set1.union(hashed_shingle_set2)
    return jacc    

In [5]:
# generate a list of all documents; ordered sets of integers (hashed shingles)
k = 7 # (de använder 9 i boken)
folder_path = 'data/'
hashed_shingle_sets = [] # length (nr od documents): 99, ranging in length between 23 and 4505 unique shingles
#universal_set = [] # length: 40094
for filename in os.listdir(folder_path):
    if filename.endswith('.txt'):
        file_path = os.path.join(folder_path, filename)
        ordered_shingles = Shingling(k, file_path)
        hashed_shingle_sets.append(ordered_shingles)
        #universal_set.extend(shingles)
        
        
#universal_set = OrderedSet(universal_set)


In [6]:
# builds minHash signatures from given set of integers (i.e. documents)
# The minhash value of any column is the number of
# the first row, in the permuted order, in which the column has a 1.
def MinHashing(sets_of_integers):
    """
    * Length of signatures, n, can be selected randomly
    * Theoretically: from the column representing set S, construct the minhash signature for S, the
      vector [h1(S), h2(S), . . . , hn(S)]
    * In practise: instead of picking n random permutations of rows, we pick n randomly
      chosen hash functions h1, h2, . . . , hn on the rows.
    * If the number of rows in the characteristic matrix is not a prime number,
      there will be collisions, where two rows get the same hash value.
    * Compression, n = 50 (att gå från 40094 till 1000 känns bättre)
    * The minHash signature can be a set or array
    """
    #enabling minHash generation: reconstruct the list of hashed_shingle_sets into a characteristic matrix:
    #dimensions: (len(universal_set), len(hashed_shingle_sets)) = (2^(32), 99) (2^32=4294967296, 2^16=65536, har 40094 unika shingles)
    characteristic_matrix = np.zeros((2**32,len(sets_of_integers)), dtype=int)
    for i in range(len(sets_of_integers)):
        characteristic_matrix[sets_of_integers[i],i] = 1
        
    #heuristic functions: 
    # ai, bi randomly set integers, p prime number
    #hi = (ai*x+bi) % p
    n=50
    scales = np.random.choice(a=10*n, size=n, replace=False)
    bias = np.random.choice(a=10*n, size=n, replace=False)
    #chose a prime number close to 10*n, n=50: 503, 509, 521
    prime_numb = 509

    #signature matrix:
    SIG = np.full((n, len(sets_of_integers)), np.inf)
    
    for r in range(len(characteristic_matrix)):
        h_r = np.zeros(n)
        for i in range(n):
            h_r[i] = (scales[i]*r+bias[i])%prime_numb
        
    for c in range(len(sets_of_integers)):
        if characteristic_matrix[r,c] == 1:
            #decide if should change value in the signature matrix:
            for i in range(n):
                if h_r[i]<SIG[i,c]:
                   SIG[i,c] = h_r[i]
                    
    return SIG

In [ ]:
SIG = MinHashing(hashed_shingle_sets)

In [ ]:
"""
Estimating the Jaccard similarity between two sets of integers:
  * the probability that two columns have the same value in a given row of
    the signature matrix equals the Jaccard similarity of the sets corresponding to
    those columns
  * the expected number of rows in
    which two columns agree equals the Jaccard similarity of their corresponding
    sets
"""
def CompareSignatures(sig_1, sig_2, n)
    similarity = len(np.intersect1d(sig_1, sig_2))/n